In [19]:
from requests_futures.sessions import FuturesSession
from tqdm import tqdm
import requests
import json
import pandas as pd

In [42]:
df = pd.read_csv(r'C:\Users\Dalton\Documents\Portfolio Projects\Automotive Price Analysis\car data\car_data_05-11-2023--080002.csv')

In [43]:
df['Make'] = ""
df['Model'] = ""
df['Year'] = ""
df['Trim'] = ""

In [44]:
url = 'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVINValuesBatch/'

chunk_size = 50  # Set chunk size to 50
session = FuturesSession(max_workers=40)  # Set the number of concurrent requests

futures = []

for i in range(0, len(df), chunk_size):
    chunk = df[i:i+chunk_size]
    vins = ';'.join(chunk['VIN'])
    post_fields = {'format': 'json', 'data': vins}
    futures.append(session.post(url, data=post_fields))

results = []

for future in tqdm(futures, total=len(futures)):
    response = future.result()
    data = json.loads(response.text)
    for result in data['Results']:
        if 'Message' in result:
            print(f"Message from API: {result['Message']}")
        else:
            results.append({
                'VIN': result['VIN'],
                'Make': result['Make'],
                'Model': result['Model'],
                'Year': result['ModelYear'],
                'Trim': result['Trim']
            })

results_df = pd.DataFrame(results)
df = pd.merge(df, results_df, on='VIN')

df.to_csv(r'C:\Users\Dalton\Documents\Portfolio Projects\Automotive Price Analysis.csv', index=False)

100%|██████████| 40/40 [00:15<00:00,  2.62it/s]
